In [ ]:
# ==============================================
# STEP 1: USER-CONFIGURABLE PARAMETERS
# ==============================================
TICKER = "AAPL"                    # Stock ticker symbol (no suffix)
FIXED_HORIZON = 10                 # Explicit forecast period in years
RISK_FREE_RATE = 0.04              # 10Y Treasury Yield
MARKET_RISK_PREMIUM = 0.0402       # Historical equity risk premium
BETA = None                        # None to fetch from API
COST_OF_DEBT = None                # None to fetch from API 
TAX_RATE = 0.21                    # Corporate tax rate
TERMINAL_GROWTH = 0.025            # Perpetual growth assumption (< WACC)
API_KEY = "Ke2DPxQbWpp1hk0BCTPqs15mS7ZmdaUT"       # Get from FinancialModelingPrep

# ==============================================
# STEP 2: DATA FETCHING & VALIDATION
# ==============================================
from financetoolkit import Toolkit
import pandas as pd
import numpy as np
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt

class DCFAnalyzer:
    def __init__(self):
        self.financials = {
            'market_cap': None, 'total_debt': None, 'shares_outstanding': None,
            'current_price': None, 'fcf': None, 'beta': None, 'cost_of_debt': None
        }
        self.wacc = None
        self.implied_growth = None
        self.intrinsic_value = None
        
        self._fetch_data()
        self._validate_data()
        self._calculate_wacc()
        
    def _fetch_data(self):
        """Retrieve financial data from FinanceToolkit with error handling"""
        try:
            companies = Toolkit([TICKER], api_key=API_KEY)
            
            # Validate API connection
            if companies.get_profile().empty:
                raise ValueError("API returned no data - check key/ticker")
            
            # Extract profile data
            profile = companies.get_profile().transpose()
            self.financials['market_cap'] = profile.loc['Market Capitalization', TICKER]
            self.financials['shares_outstanding'] = profile.loc['Shares Outstanding', TICKER]
            self.financials['current_price'] = profile.loc['Price', TICKER]
            
            # Extract balance sheet data
            balance = companies.get_balance_sheet_statement()
            self.financials['total_debt'] = balance.loc['Total Debt', TICKER].iloc[-1]
            
            # Extract cash flow data
            cashflow = companies.get_cash_flow_statement()
            self.financials['fcf'] = cashflow.loc['Free Cash Flow', TICKER].iloc[-1]
            
            # Calculate beta if not provided
            self.financials['beta'] = (
                companies.ratios.collect_valuation_ratios().loc['Beta', TICKER] 
                if BETA is None else BETA
            )
            
            # Calculate cost of debt if not provided
            self.financials['cost_of_debt'] = (
                companies.ratios.get_interest_coverage_ratio().mean().loc[TICKER]
                if COST_OF_DEBT is None else COST_OF_DEBT
            )
            
        except Exception as e:
            print(f"❌ Data retrieval failed: {str(e)}")
            exit()

    def _validate_data(self):
        """Ensure all critical financial metrics are valid"""
        required_values = [
            self.financials['market_cap'],
            self.financials['total_debt'],
            self.financials['shares_outstanding'],
            self.financials['current_price'],
            self.financials['fcf']
        ]
        
        if any(v <= 0 for v in required_values):
            print("❌ Invalid financial values detected:")
            print(f"Market Cap: ${self.financials['market_cap']/1e9:.2f}B")
            print(f"Total Debt: ${self.financials['total_debt']/1e9:.2f}B")
            print(f"Shares Outstanding: {self.financials['shares_outstanding']/1e6:.2f}M")
            print(f"Current Price: ${self.financials['current_price']:.2f}")
            print(f"FCF: ${self.financials['fcf']/1e9:.2f}B")
            exit()

    # ==============================================
    # STEP 3: WACC CALCULATION (DUAL METHODS)
    # ==============================================
    def _calculate_wacc(self):
        """Calculate WACC using manual and FTK methods"""
        def manual_wacc():
            try:
                equity = self.financials['market_cap']
                debt = self.financials['total_debt']
                total_capital = equity + debt
                
                cost_equity = RISK_FREE_RATE + self.financials['beta'] * MARKET_RISK_PREMIUM
                cost_debt = self.financials['cost_of_debt'] * (1 - TAX_RATE)
                
                return (equity/total_capital)*cost_equity + (debt/total_capital)*cost_debt
            except Exception as e:
                print(f"⚠️ Manual WACC failed: {str(e)}")
                return None
        
        def ftk_wacc():
            try:
                companies = Toolkit([TICKER], api_key=API_KEY)
                return companies.ratios.get_wacc().iloc[0][0]
            except Exception as e:
                print(f"⚠️ FTK WACC failed: {str(e)}")
                return None
        
        self.wacc = ftk_wacc() or manual_wacc()
        
        if not self.wacc or self.wacc <= TERMINAL_GROWTH:
            print(f"❌ Invalid WACC: {self.wacc:.2%} (must > {TERMINAL_GROWTH:.2%})")
            exit()

    # ==============================================
    # STEP 4: DCF VALUATION MODEL
    # ==============================================
    def calculate_intrinsic_value(self, growth_rate: float) -> float:
        """Calculate per-share intrinsic value for given growth rate"""
        try:
            terminal_cf = self.financials['fcf'] * (1 + growth_rate)**FIXED_HORIZON
            terminal_value = terminal_cf * (1 + TERMINAL_GROWTH) / (self.wacc - TERMINAL_GROWTH)
            
            explicit_cf = sum(
                self.financials['fcf'] * (1 + growth_rate)**y / (1 + self.wacc)**y 
                for y in range(1, FIXED_HORIZON + 1)
            )
            
            return (explicit_cf + terminal_value) / self.financials['shares_outstanding']
        except Exception as e:
            print(f"❌ DCF calculation failed: {str(e)}")
            exit()

    # ==============================================
    # STEP 5: REVERSE DCF (PRICED-IN EXPECTATIONS)
    # ==============================================
    def calculate_implied_growth(self):
        """Determine growth rate implied by current market price"""
        def optimization_target(growth_rate: float) -> float:
            intrinsic_value = self.calculate_intrinsic_value(growth_rate)
            return (intrinsic_value - self.financials['current_price'])**2
        
        result = minimize_scalar(
            optimization_target,
            bounds=(-0.10, 0.50),
            method='bounded'
        )
        
        if not result.success:
            print(f"⚠️ Reverse DCF failed: {result.message}")
            return None
            
        return result.x

    # ==============================================
    # STEP 6: VISUALIZATION & OUTPUT
    # ==============================================
    def generate_report(self):
        """Create comprehensive output report"""
        # Calculate key metrics
        self.intrinsic_value = self.calculate_intrinsic_value(0.05)  # 5% growth baseline
        self.implied_growth = self.calculate_implied_growth()
        
        # Generate sensitivity analysis
        growth_rates = np.linspace(-0.05, 0.15, 100)
        values = [self.calculate_intrinsic_value(g) for g in growth_rates]
        
        # Create plot
        plt.figure(figsize=(12, 7))
        plt.plot(growth_rates, values, label='DCF Valuation', linewidth=2)
        plt.axhline(self.financials['current_price'], color='r', linestyle='--', 
                   label='Current Price', linewidth=2)
        plt.axvline(self.implied_growth, color='g', linestyle=':', 
                   label='Implied Growth', linewidth=2)
        
        plt.title(f'DCF Sensitivity Analysis: {TICKER}', fontsize=14)
        plt.xlabel('Free Cash Flow Growth Rate', fontsize=12)
        plt.ylabel('Intrinsic Value per Share ($)', fontsize=12)
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'{TICKER}_dcf_analysis.png', dpi=300, bbox_inches='tight')
        
        # Create results DataFrame
        results = pd.DataFrame({
            'Metric': [
                'Current Price', 'Implied Growth Rate', 'WACC',
                'Terminal Growth Rate', '5% Growth Intrinsic Value',
                'Market Cap', 'Total Debt', 'Beta', 'Cost of Debt'
            ],
            'Value': [
                f"${self.financials['current_price']:.2f}",
                f"{self.implied_growth:.2%}",
                f"{self.wacc:.2%}",
                f"{TERMINAL_GROWTH:.2%}",
                f"${self.intrinsic_value:.2f}",
                f"${self.financials['market_cap']/1e9:.2f}B",
                f"${self.financials['total_debt']/1e9:.2f}B",
                f"{self.financials['beta']:.2f}",
                f"{self.financials['cost_of_debt']:.2%}"
            ]
        })
        
        # Save and display results
        results.to_csv(f'{TICKER}_dcf_results.csv', index=False)
        print(results.to_string(index=False))
        plt.show()

# ==============================================
# MAIN EXECUTION
# ==============================================
if __name__ == "__main__":
    analyzer = DCFAnalyzer()
    analyzer.generate_report()


Obtaining company profiles: 100%|██████████| 1/1 [00:00<00:00,  9.70it/s]


❌ Data retrieval failed: 'AAPL'


TypeError: '<=' not supported between instances of 'NoneType' and 'int'

: 

Obtaining company profiles: 100%|██████████| 1/1 [00:00<00:00,  9.79it/s]


Company Name: Apple Inc.


Obtaining historical statistics: 100%|██████████| 1/1 [00:00<00:00,  9.75it/s]


TypeError: Index must be a MultiIndex